In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
nb_batches = 4
in_chan, out_chan = 2, 6
in_dim = (8,)
k_size = (3,)
dtype = torch.float32


input_1d = torch.rand((nb_batches, in_chan, *in_dim), dtype=dtype)
kernel_1d = torch.rand((out_chan, in_chan, *k_size), dtype=dtype)

in_dim = (8,) * 2
k_size = (3,) * 2

input_2d = torch.rand((nb_batches, in_chan, *in_dim), dtype=dtype)
kernel_2d = torch.rand((out_chan, in_chan, *k_size), dtype=dtype)

in_dim = (8,) * 3
k_size = (3,) * 3

input_3d = torch.rand((nb_batches, in_chan, *in_dim), dtype=dtype)
kernel_3d = torch.rand((out_chan, in_chan, *k_size), dtype=dtype)

In [ ]:
print(input_2d.shape, kernel_2d.shape)
# By batch, for each window, flatten / unfold the matmul to perform
# B * I * H * W --> B * (I * K1 * K2) * -1
tmp_in = F.unfold(input_2d, kernel_2d.shape[-2:])
print(tmp_in.shape)

In [ ]:
print(tmp_in.shape)
# B * ops_per_window * num_windows --> B * num_windows * ops_per_window
tmp_in = tmp_in.transpose(1, 2)
print(tmp_in.shape)

In [ ]:
print(kernel_2d.shape)
# O * I * K1 * K2 --> O * ops_per_window
tmp_k = kernel_2d.view(kernel_2d.shape[0], -1)
print(tmp_k.shape)

In [ ]:
print(tmp_k.shape)
# O * ops_per_window --> ops_per_window * O
tmp_k = tmp_k.t()
print(tmp_k.shape)

In [ ]:
print(tmp_in.shape, tmp_k.shape)
# B * num_windows * ops_per_window @ ops_per_window * O --> B * num_windows * O
tmp = tmp_in.matmul(tmp_k)
print(tmp.shape)

In [ ]:
print(tmp.shape)
# B * num_windows * O --> B * O * num_windows
tmp = tmp.transpose(1, 2)
print(tmp.shape)

In [ ]:
print(tmp.shape)
# B * O * num_windows --> B * O * H * W
tmp = tmp.view(input_2d.shape[0], kernel_2d.shape[0], input_2d.shape[-2] - (kernel_2d.shape[-2] - 1), input_2d.shape[-1] - (kernel_2d.shape[-1] - 1))
print(tmp.shape)

In [ ]:
res = F.unfold(input_2d, kernel_2d.shape[-2:]).transpose(1, 2).matmul(kernel_2d.view(kernel_2d.shape[0], -1).t()).transpose(1, 2).view(input_2d.shape[0], kernel_2d.shape[0], input_2d.shape[-2] - (kernel_2d.shape[-2] - 1), input_2d.shape[-1] - (kernel_2d.shape[-1] - 1))
print(res.shape)

In [ ]:
%timeit -n 10 _ = F.unfold(input_2d, kernel_2d.shape[-2:]).transpose(1, 2).matmul(kernel_2d.view(kernel_2d.shape[0], -1).t()).transpose(1, 2).view(input_2d.shape[0], kernel_2d.shape[0], input_2d.shape[-2] - (kernel_2d.shape[-2] - 1), input_2d.shape[-1] - (kernel_2d.shape[-1] - 1))

## SlidingND

In [ ]:
def naive_sliding1D(fn, input, kernel, padding=1, dtype=torch.float32):
    """Apply fn in a convolutioned fashion
    
    Args:
        fn (callable)
        input (torch.Tensor[N, I, L]):
        kernel (torch.Tensor[O, I, K]):
        padding (int, optional):

    Returns:
        torch.Tensor[N, O, L]
    """
    
    if isinstance(padding, int):
        padding = (padding, ) * 2 * (kernel.ndim - 2)
    
    if input.ndim != kernel.ndim:
        raise AssertionError(f"expected (2+N)D input, received {input.ndim}")
    
    # N, O, ...
    res = torch.zeros((input.shape[0], kernel.shape[0], *input.shape[2:]), dtype=dtype)
    _pad = F.pad(input, padding, mode='constant', value=0.)
    
    for bidx in range(res.shape[0]):
        for oidx in range(kernel.shape[0]):
            for idx in range(res.shape[-1]):
                res[bidx, oidx, idx] = fn(_pad[bidx, ..., idx: idx + kernel.shape[-1]], kernel[oidx, ...])

    return res

In [ ]:
def mid_sliding1D(fn, input, kernel, padding=1, dtype=torch.float32):
    """Apply fn in a convolutioned fashion
    
    Args:
        fn (callable)
        input (torch.Tensor[N, I, L]):
        kernel (torch.Tensor[O, I, K]):
        padding (int, optional):

    Returns:
        torch.Tensor[N, O, L]
    """
    
    if isinstance(padding, int):
        padding = (padding, ) * 2 * (kernel.ndim - 2)
    
    if input.ndim != kernel.ndim:
        raise AssertionError(f"expected (2+N)D input, received {input.ndim}")
    
    # N, O, ...
    res = torch.zeros((input.shape[0], kernel.shape[0], *input.shape[2:]), dtype=dtype)
    _pad = F.pad(input, padding, mode='constant', value=0.)
    
    for idx in range(res.shape[-1]):
        # N, O <-- (N, I, K - O, I, K)
        res[..., idx] = fn(_pad[..., idx: idx + kernel.shape[-1]], kernel)

    return res

In [ ]:
def sliding1D(fn, input, kernel, padding=1, dtype=torch.float32):
    """Apply fn in a convolutioned fashion
    
    Args:
        fn (callable)
        input (torch.Tensor[N, I, L]):
        kernel (torch.Tensor[O, I, K]):
        padding (int, optional):

    Returns:
        torch.Tensor[N, O, L]
    """
    
    if isinstance(padding, int):
        padding = (padding, ) * 2 * (kernel.ndim - 2)
    
    if input.ndim != kernel.ndim:
        raise AssertionError(f"expected (2+N)D input, received {input.ndim}")
    
    # N, O, ...
    res = torch.zeros((input.shape[0], kernel.shape[0], *input.shape[2:]), dtype=dtype)
    _pad = F.pad(input, padding, mode='constant', value=0.)
    
    for kidx in range(kernel.shape[-1]):
        # N, O, L <-- N, I, L @ O, I
        res += fn(_pad[..., kidx: kidx + input.shape[-1]], kernel[..., kidx])

    return res

In [ ]:
def naive_sliding2D(fn, input, kernel, padding=1, dtype=torch.float32):
    """Apply fn in a convolutioned fashion
    
    Args:
        fn (callable)
        input (torch.Tensor[N, I, H, W]):
        kernel (torch.Tensor[O, I, K1, K2]):
        padding (int, optional):

    Returns:
        torch.Tensor[N, O, H, W]
    """
    
    if isinstance(padding, int):
        padding = (padding, ) * 2 * (kernel.ndim - 2)
    
    if input.ndim != kernel.ndim:
        raise AssertionError(f"expected (2+N)D input, received {input.ndim}")
    
    # N, O, ...
    res = torch.zeros((input.shape[0], kernel.shape[0], *input.shape[2:]), dtype=dtype)
    _pad = F.pad(input, padding, mode='constant', value=0.)
    
    for bidx in range(res.shape[0]):
        for oidx in range(kernel.shape[0]):
            for row in range(res.shape[-2]):
                for col in range(res.shape[-1]):
                    # N, O <-- (N, I, H, W - O, I, K1, K2)
                    res[bidx, oidx, row, col] = fn(_pad[bidx, ..., row: row + kernel.shape[-2], col: col + kernel.shape[-1]],
                                                   kernel[oidx, ...])

    return res

In [ ]:
def mid_sliding2D(fn, input, kernel, padding=1, dtype=torch.float32):
    """Apply fn in a convolutioned fashion
    
    Args:
        fn (callable)
        input (torch.Tensor[N, I, H, W]):
        kernel (torch.Tensor[O, I, K1, K2]):
        padding (int, optional):

    Returns:
        torch.Tensor[N, O, H, W]
    """
    
    if isinstance(padding, int):
        padding = (padding, ) * 2 * (kernel.ndim - 2)
    
    if input.ndim != kernel.ndim:
        raise AssertionError(f"expected (2+N)D input, received {input.ndim}")
    
    # N, O, ...
    res = torch.zeros((input.shape[0], kernel.shape[0], *input.shape[2:]), dtype=dtype)
    _pad = F.pad(input, padding, mode='constant', value=0.)
    
    # Loop on input spatially
    for row in range(res.shape[-2]):
        for col in range(res.shape[-1]):
            # N, O <-- (N, I, H, W - O, I, K1, K2)
            res[..., row, col] = fn(_pad[..., row: row + kernel.shape[-2], col: col + kernel.shape[-1]],
                                    kernel)

    return res

In [ ]:
def sliding2D(fn, input, kernel, padding=1, dtype=torch.float32):
    """Apply fn in a convolutioned fashion
    
    Args:
        fn (callable)
        input (torch.Tensor[N, I, H, W]):
        kernel (torch.Tensor[O, I, K1, K2]):
        padding (int, optional):

    Returns:
        torch.Tensor[N, O, H, W]
    """
    
    if isinstance(padding, int):
        padding = (padding, ) * 2 * (kernel.ndim - 2)
    
    if input.ndim != kernel.ndim:
        raise AssertionError(f"expected (2+N)D input, received {input.ndim}")
    
    # N, O, ...
    res = torch.zeros((input.shape[0], kernel.shape[0], *input.shape[2:]), dtype=dtype)
    _pad = F.pad(input, padding, mode='constant', value=0.)
    
    # Loop on kernel spatially
    for krow in range(kernel.shape[-2]):
        for kcol in range(kernel.shape[-1]):
            # N, O, ... <-- N, I, H, W @ O, I
            res += fn(_pad[..., krow: krow + input.shape[-2], kcol: kcol + input.shape[-1]], kernel[..., krow, kcol])

    return res

In [ ]:
def naive_sliding3D(fn, input, kernel, padding=1, dtype=torch.float32):
    """Apply fn in a convolutioned fashion
    
    Args:
        fn (callable)
        input (torch.Tensor[N, I, H, W, D]):
        kernel (torch.Tensor[O, I, K1, K2, K3]):
        padding (int, optional):

    Returns:
        torch.Tensor[N, O, H, W, D]
    """
    
    if isinstance(padding, int):
        padding = (padding, ) * 2 * (kernel.ndim - 2)
    
    if input.ndim != kernel.ndim:
        raise AssertionError(f"expected (2+N)D input, received {input.ndim}")
    
    # N, O, ...
    res = torch.zeros((input.shape[0], kernel.shape[0], *input.shape[2:]), dtype=dtype)
    _pad = F.pad(input, padding, mode='constant', value=0.)
    
    for bidx in range(res.shape[0]):
        for oidx in range(kernel.shape[0]):
            for row in range(res.shape[-3]):
                for col in range(res.shape[-2]):
                    for depth in range(res.shape[-1]):
                        # . <-- (I, K1, K2, K3 - I, K1, K2, K3)
                        res[bidx, oidx, row, col] = fn(_pad[bidx, ..., row: row + kernel.shape[-3], col: col + kernel.shape[-2], depth: depth + kernel.shape[-1]],
                                                       kernel[oidx, ...])

    return res

In [ ]:
def mid_sliding3D(fn, input, kernel, padding=1, dtype=torch.float32):
    """Apply fn in a convolutioned fashion
    
    Args:
        fn (callable)
        input (torch.Tensor[N, I, H, W, D]):
        kernel (torch.Tensor[O, I, K1, K2, K3]):
        padding (int, optional):

    Returns:
        torch.Tensor[N, O, H, W, D]
    """
    
    if isinstance(padding, int):
        padding = (padding, ) * 2 * (kernel.ndim - 2)
    
    if input.ndim != kernel.ndim:
        raise AssertionError(f"expected (2+N)D input, received {input.ndim}")
    
    # N, O, ...
    res = torch.zeros((input.shape[0], kernel.shape[0], *input.shape[2:]), dtype=dtype)
    _pad = F.pad(input, padding, mode='constant', value=0.)
    
    # Loop on input spatially
    for row in range(res.shape[-3]):
        for col in range(res.shape[-2]):
            for depth in range(res.shape[-1]):
                # N, O <-- (N, I, H, W, D - O, I, K1, K2, K3)
                res[..., row, col, depth] = fn(_pad[..., row: row + kernel.shape[-3], col: col + kernel.shape[-2], depth: depth + kernel.shape[-1]],
                                        kernel)

    return res

In [ ]:
def sliding3D(fn, input, kernel, padding=1, dtype=torch.float32):
    """Apply fn in a convolutioned fashion
    
    Args:
        fn (callable)
        input (torch.Tensor[N, I, H, W, D]):
        kernel (torch.Tensor[O, I, K1, K2, K3]):
        padding (int, optional):

    Returns:
        torch.Tensor[N, O, H, W, D]
    """
    
    if isinstance(padding, int):
        padding = (padding, ) * 2 * (kernel.ndim - 2)
    
    if input.ndim != kernel.ndim:
        raise AssertionError(f"expected (2+N)D input, received {input.ndim}")
    
    # N, O, ...
    res = torch.zeros((input.shape[0], kernel.shape[0], *input.shape[2:]), dtype=dtype)
    _pad = F.pad(input, padding, mode='constant', value=0.)
    
    # Loop on kernel spatially
    for krow in range(kernel.shape[-3]):
        for kcol in range(kernel.shape[-2]):
            for kdep in range(kernel.shape[-1]):
                # N, O, ... <-- N, I, H, W @ O, I
                res += fn(_pad[..., krow: krow + input.shape[-3], kcol: kcol + input.shape[-2], kdep: kdep + input.shape[-1]],
                          kernel[..., krow, kcol, kdep])

    return res

## OpND

In [ ]:
def naive_opND(fn, input, kernel):
    """Apply Conv1D locally
    
    Args:
        input (torch.Tensor[I, K1, ..., Kn]):
        kernel (torch.Tensor[I, K1, ..., Kn]):

    Returns:
        torch.Tensor[]
    """

    if input.shape != kernel.shape:
        raise AssertionError("expected input and kernel to have identical shape")
    
    return fn(input, kernel)

In [ ]:
def mid_opND(fn, input, kernel):
    """Apply Conv1D locally
    
    Args:
        input (torch.Tensor[N, I, K1, ..., Kn]):
        kernel (torch.Tensor[O, I, K1, ..., Kn]):

    Returns:
        torch.Tensor[N, O]
    """

    if input.ndim != kernel.ndim:
        raise AssertionError("wrong number of dimensions")
    if input.shape[1:] != kernel.shape[1:]:
        raise AssertionError("wrong shapes")
    
    return fn(input.unsqueeze(1), kernel.unsqueeze(0))

In [ ]:
def opND(fn, input, kernel, n):
    """Apply Conv1D locally
    
    Args:
        input (torch.Tensor[N, I, ...]):
        kernel (torch.Tensor[O, I]):

    Returns:
        torch.Tensor[N, O, ...]
    """
    
    if input.ndim != kernel.ndim + n:
        raise AssertionError("wrong number of dimensions")
    if input.shape[1] != kernel.shape[1]:
        raise AssertionError("expected input and kernel to share same second axis size")
    
    return fn(input.unsqueeze(1), kernel.unsqueeze(0)[(...,) + (None,) * n])

## ConvND

In [ ]:
def n_convND(a, b):
    """Apply ConvND locally
    
    Args:
        a (torch.Tensor[I, K1, ..., Kn]):
        b (torch.Tensor[I, K1, ..., Kn]):

    Returns:
        torch.Tensor[1]
    """
    return a.mul(b).sum()


def m_convND(a, b):
    """Apply ConvND locally
    
    Args:
        a (torch.Tensor[N, O, I, K1, ..., Kn]):
        b (torch.Tensor[N, O, I, K1, ..., Kn]):

    Returns:
        torch.Tensor[N, O]
    """
    return a.mul(b).flatten(2).sum(2)


def convND(a, b):
    """Apply ConvND locally
    
    Args:
        a (torch.Tensor[N, 1, I, ...]):
        b (torch.Tensor[1, O, I, ...]):

    Returns:
        torch.Tensor[N, O, ...]
    """
    return a.mul(b).sum(2)

In [ ]:
def naive_convND(input, kernel):
    return naive_opND(n_convND, input, kernel)

def mid_convND(input, kernel):
    return mid_opND(m_convND, input, kernel)

def conv1D(input, kernel):
    return opND(convND, input, kernel, n=1)

In [ ]:
%timeit -n 10 _ = naive_sliding1D(naive_convND, input_1d, kernel_1d)
%timeit -n 10 _ = mid_sliding1D(mid_convND, input_1d, kernel_1d)
%timeit -n 10 _ = sliding1D(conv1D, input_1d, kernel_1d)
%timeit -n 10 _ = F.conv1d(input_1d, kernel_1d, padding=1)

In [ ]:
def conv2D(input, kernel):
    return opND(convND, input, kernel, n=2)

In [ ]:
%timeit -n 10 _ = naive_sliding2D(naive_convND, input_2d, kernel_2d)
%timeit -n 10 _ = mid_sliding2D(mid_convND, input_2d, kernel_2d)
%timeit -n 10 _ = sliding2D(conv2D, input_2d, kernel_2d)
%timeit -n 10 _ = F.conv2d(input_2d, kernel_2d, padding=1)

In [ ]:
def conv3D(input, kernel):
    return opND(convND, input, kernel, n=3)

In [ ]:
%timeit -n 10 _ = naive_sliding3D(naive_convND, input_3d, kernel_3d)
%timeit -n 10 _ = mid_sliding3D(mid_convND, input_3d, kernel_3d)
%timeit -n 10 _ = sliding3D(conv3D, input_3d, kernel_3d)
%timeit -n 10 _ = F.conv3d(input_3d, kernel_3d, padding=1)

## AdderND

In [ ]:
def n_adderND(a, b):
    """Apply ConvND locally
    
    Args:
        a (torch.Tensor[I, K1, ..., Kn]):
        b (torch.Tensor[I, K1, ..., Kn]):

    Returns:
        torch.Tensor[1]
    """
    return a.sub(b).abs_().sum()


def m_adderND(a, b):
    """Apply ConvND locally
    
    Args:
        a (torch.Tensor[N, O, I, K1, ..., Kn]):
        b (torch.Tensor[N, O, I, K1, ..., Kn]):

    Returns:
        torch.Tensor[N, O]
    """
    return a.sub(b).abs_().flatten(2).sum(2)


def adderND(a, b):
    """Apply AdderND locally
    
    Args:
        a (torch.Tensor[N, 1, I, ...]):
        b (torch.Tensor[1, O, I, ...]):

    Returns:
        torch.Tensor[N, O, ...]
    """

    return a.sub(b).abs_().sum(2)

In [ ]:
def naive_adderND(input, kernel):
    return naive_opND(n_adderND, input, kernel)

def mid_adderND(input, kernel):
    return mid_opND(m_adderND, input, kernel)

def adder1D(input, kernel):
    return opND(adderND, input, kernel, n=1)

In [ ]:
%timeit -n 10 _ = naive_sliding1D(naive_adderND, input_1d, kernel_1d)
%timeit -n 10 _ = mid_sliding1D(mid_adderND, input_1d, kernel_1d)
%timeit -n 10 _ = sliding1D(conv1D, input_1d, kernel_1d)

In [ ]:
def adder2D(input, kernel):
    return opND(adderND, input, kernel, n=2)

In [ ]:
%timeit -n 10 _ = naive_sliding2D(naive_adderND, input_2d, kernel_2d)
%timeit -n 10 _ = mid_sliding2D(mid_adderND, input_2d, kernel_2d)
%timeit -n 10 _ = sliding2D(adder2D, input_2d, kernel_2d)

In [ ]:
def adder3D(input, kernel):
    return opND(adderND, input, kernel, n=3)

In [ ]:
%timeit -n 10 _ = naive_sliding3D(naive_adderND, input_3d, kernel_3d)
%timeit -n 10 _ = mid_sliding3D(mid_adderND, input_3d, kernel_3d)
%timeit -n 10 _ = sliding3D(adder3D, input_3d, kernel_3d)

## CosimND

In [ ]:
def n_cosimND(a, b, q=0):
    """Apply ConvND locally
    
    Args:
        a (torch.Tensor[I, K1, ..., Kn]):
        b (torch.Tensor[I, K1, ..., Kn]):

    Returns:
        torch.Tensor[1]
    """
    return n_convND(a, b).div_(a.norm().add_(q) * b.norm().add_(q))


def m_cosimND(a, b, q=0):
    """Apply ConvND locally
    
    Args:
        a (torch.Tensor[N, O, I, K1, ..., Kn]):
        b (torch.Tensor[N, O, I, K1, ..., Kn]):

    Returns:
        torch.Tensor[N, O]
    """
    return m_convND(a, b).div_(a.pow(2).flatten(2).sum(2).add_(q) * b.pow(2).flatten(2).sum(2).add_(q))

def cosimND(a, b, q=0):
    """Apply CosimND locally
    
    Args:
        a (torch.Tensor[N, 1, I, ...]):
        b (torch.Tensor[1, O, I, ...]):

    Returns:
        torch.Tensor[N, O, ...]
    """

    return convND(a, b).div_(a.norm(dim=2).add_(q) * b.norm(dim=2).add_(q))

In [ ]:
def naive_cosimND(input, kernel):
    return naive_opND(n_cosimND, input, kernel)

def mid_cosimND(input, kernel):
    return mid_opND(m_cosimND, input, kernel)

def cosim1D(input, kernel):
    return opND(cosimND, input, kernel, n=1)

In [ ]:
%timeit -n 10 _ = naive_sliding1D(naive_cosimND, input_1d, kernel_1d)
%timeit -n 10 _ = mid_sliding1D(mid_cosimND, input_1d, kernel_1d)
%timeit -n 10 _ = sliding1D(cosim1D, input_1d, kernel_1d)

In [ ]:
def cosim2D(input, kernel):
    return opND(cosimND, input, kernel, n=2)

In [ ]:
%timeit -n 10 _ = naive_sliding2D(naive_cosimND, input_2d, kernel_2d)
%timeit -n 10 _ = mid_sliding2D(mid_cosimND, input_2d, kernel_2d)
%timeit -n 10 _ = sliding2D(cosim2D, input_2d, kernel_2d)

In [ ]:
def cosim3D(input, kernel):
    return opND(cosimND, input, kernel, n=3)

In [ ]:
%timeit -n 10 _ = naive_sliding3D(naive_cosimND, input_3d, kernel_3d)
%timeit -n 10 _ = mid_sliding3D(mid_cosimND, input_3d, kernel_3d)
%timeit -n 10 _ = sliding3D(cosim3D, input_3d, kernel_3d)

## SharpCosimND

In [ ]:
def n_scosimND(a, b, p=2, q=1e-3):
    """Apply ConvND locally
    
    Args:
        a (torch.Tensor[I, K1, ..., Kn]):
        b (torch.Tensor[I, K1, ..., Kn]):

    Returns:
        torch.Tensor[1]
    """
    num = n_convND(a, b)
    return torch.sign(num) * num.div_(a.norm().add_(q) * b.norm().add_(q)).pow_(p)


def m_scosimND(a, b, p=2, q=1e-3):
    """Apply ConvND locally
    
    Args:
        a (torch.Tensor[N, O, I, K1, ..., Kn]):
        b (torch.Tensor[N, O, I, K1, ..., Kn]):

    Returns:
        torch.Tensor[N, O]
    """
    num = m_convND(a, b)
    return torch.sign(num) * num.div_(a.pow(2).flatten(2).sum(2).add_(q) * b.pow(2).flatten(2).sum(2).add_(q)).pow_(p)


def scosimND(a, b, p=2, q=1e-3):
    """Apply Conv1D locally
    
    Args:
        a (torch.Tensor[N, 1, I, ...]):
        b (torch.Tensor[1, O, I, ...]):

    Returns:
        torch.Tensor[N, O, ...]
    """

    num = convND(a, b)
    return torch.sign(num) * num.div_(a.norm(dim=2).add_(q) * b.norm(dim=2).add_(q)).pow_(p)

In [ ]:
def naive_scosimND(input, kernel):
    return naive_opND(n_scosimND, input, kernel)

def mid_scosimND(input, kernel):
    return mid_opND(m_scosimND, input, kernel)

def scosim1D(input, kernel):
    return opND(scosimND, input, kernel, n=1)

In [ ]:
%timeit -n 10 _ = naive_sliding1D(naive_scosimND, input_1d, kernel_1d)
%timeit -n 10 _ = mid_sliding1D(mid_scosimND, input_1d, kernel_1d)
%timeit -n 10 _ = sliding1D(scosim1D, input_1d, kernel_1d)

In [ ]:
def scosim2D(input, kernel):
    return opND(scosimND, input, kernel, n=2)

In [ ]:
%timeit -n 10 _ = naive_sliding2D(naive_scosimND, input_2d, kernel_2d)
%timeit -n 10 _ = mid_sliding2D(mid_scosimND, input_2d, kernel_2d)
%timeit -n 10 _ = sliding2D(scosim2D, input_2d, kernel_2d)

In [ ]:
def scosim3D(input, kernel):
    return opND(scosimND, input, kernel, n=3)

In [ ]:
%timeit -n 10 _ = naive_sliding3D(naive_scosimND, input_3d, kernel_3d)
%timeit -n 10 _ = mid_sliding3D(mid_scosimND, input_3d, kernel_3d)
%timeit -n 10 _ = sliding3D(scosim3D, input_3d, kernel_3d)